# Make ChatGPT-replica!!

"학습을 시작하기 전에" 파트를 반드시 실행시켜 주세요.

각 파트 사이사이 마다 raise Exception 구문이 있기 때문에, 한 파트만 학습하고 싶은 경우에도 그냥 이후 셀 실행 하면 됩니다.

한 번에 1~3번을 모두 수행하고 싶은 경우, 중간중간에 있는 에러 구문을 제거해주시기 바랍니다.

# 학습을 시작하기 전에

### Google Drive에 모델 저장하기
한 번 학습한 모델을 재활용하기 위해 google drive에 저장

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 상수, 경로 설정 부분

In [ ]:
# Pretrained model 이름(hugging face) or 체크포인트 저장 경로
SFT_MODEL_NAME = './drive/MyDrive/Models/final_SFT_F'

SFT_DATASET_PATH = './drive/MyDrive/Models/instagram_chatgpt_F.jsonl'
SFT_MODEL_PATH = './drive/MyDrive/Models/final_SFT_F'
SFT_MODEL_OUTPUT_PATH = './drive/MyDrive/Models/final_SFT_F'
# SFT 모델을 학습시킬 때 샘플링 여부. 
# 샘플링하지 않고 모든 데이터를 학습시킬 경우 0으로 두기
SFT_SAMPLE_DATA_SIZE = 0

RM_DATASET_PATH = './drive/MyDrive/Models/kakao_cleaned_rm.jsonl'
RM_MODEL_PATH = './drive/MyDrive/Models/final_RM'
RM_MODEL_OUTPUT_PATH = './drive/MyDrive/Models/final_RM'

PPO_DATASET_PATH = './drive/MyDrive/Models/facebook_ppo.jsonl'
PPO_MODEL_PATH = './drive/MyDrive/Models/final_PPO_light_F'
PPO_MODEL_OUTPUT_PATH = './drive/MyDrive/Models/final_PPO_light_F'

# Inference 시 질문 목록
PROMPT_LIST = [
  '나 기분 좋아서 옷 샀어',
	'나 오늘 좋은 일 있어서 신발 샀어',
	'나 기분 안 좋아서 쇼핑다녀왔어',
	'너 기분 안 좋아보여서 아이스크림 사왔어',
	'나 돈 모아서 컴퓨터 샀어',
	'나 학교 가는 길에 교통사고 났어',
	'나 기분 안 좋아서 머리 잘랐어',
	'요즘 취업 준비하느라 힘들어',
	'오늘 늦게일어나서 대충 머리감고 나왔어',
	'걔가 너 싫어한대',
	'나 배탈 난 것 같아',
	'나 감기 걸린 것 같아',
	'코로나 걸렸어',
	'시험에 떨어졌어',
	'우울해서 아이패드 샀어',
	'나 부모님한테 혼났어',
	'이러면 아무도 너 안 좋아해',
]

## import modules

In [ ]:
## setup(1min)
# torch 버전 다운. torch>=2.0 에선 colosalai가 동작안함
!pip uninstall torch -y
!pip install torch==1.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))

print("cudnn version:{}".format(torch.backends.cudnn.version()))


# for ColossalAI
!pip install colossalai==0.2.7

# setup data
!git clone https://github.com/airobotlab/KoChatGPT

# setup library
!pip install openai
!pip install langchain==0.0.113
!pip install pandas>=1.4.1
!pip install transformers==4.28.0
!pip install tqdm
!pip install datasets
!pip install loralib
!pip install langchain

Found existing installation: torch 2.0.1+cu118
Uninstalling torch-2.0.1+cu118:
  Successfully uninstalled torch-2.0.1+cu118
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 897.4 kB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1+cu116 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.1+cu116 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1+cu116 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 1.13.1+cu116 which is incompatible.
Torch version:1.13.1+cu116
cuda version: 11.6
cudnn version:8302
Looking in indexes: http

Cloning into 'KoChatGPT'...
remote: Enumerating objects: 240, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 240 (delta 63), reused 84 (delta 45), pack-reused 129
Receiving objects: 100% (240/240), 11.57 MiB | 18.31 MiB/s, done.
Resolving deltas: 100% (93/93), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.0/396.0 kB 24.4

## 코랩 안꺼지게 하기

function ClickConnect() {
    var buttons = document.querySelectorAll("colab-dialog.yes-no-dialog paper-button#cancel"); 
    buttons.forEach(function(btn) { 
        btn.click(); 
    }); 
    console.log("1분마다 자동 재연결"); 
    document.querySelector("colab-toolbar-button#connect").click(); 
} 
setInterval(ClickConnect,1000*60);

function CleanCurrentOutput(){ 
    var btn = document.querySelector(".output-icon.clear_outputs_enabled.output-icon-selected[title$='현재 실행 중...'] iron-icon[command=clear-focused-or-selected-outputs]"); 
    if(btn) { console.log("30분마다 출력 지우기");
     btn.click(); 
    } 
} 
setInterval(CleanCurrentOutput,1000*60*30);

# 1. Supervised Fine Tuning

## import

In [ ]:
# import
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
from copy import deepcopy
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer
import pandas as pd
import argparse
import copy
import logging
import json
from dataclasses import dataclass, field

def safe_save_model_for_hf_trainer(trainer: transformers.Trainer, output_dir: str):
    """Collects the state dict and dump to disk."""
    state_dict = trainer.model.state_dict()
    if trainer.args.should_save:
        cpu_state_dict = {key: value.cpu() for key, value in list(state_dict.items())}
        del state_dict
        trainer._save(output_dir, state_dict=cpu_state_dict)  # noqa

## hyperparameter 설정

In [ ]:
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--data_path_1_SFT', type=str, default=SFT_DATASET_PATH)
parser.add_argument('--model_name', type=str, default='gpt2')
parser.add_argument('--max_epochs', type=int, default=5)
parser.add_argument('--train_batch_size', type=int, default=16)
parser.add_argument('--output_dir', type=str, default=SFT_MODEL_OUTPUT_PATH)
parser.add_argument('--save_steps', type=int, default=5000)

args = parser.parse_args(args=[])

# for test
args.model_name = SFT_MODEL_NAME  # SK GPT2, https://github.com/SKT-AI/KoGPT2

args.max_epochs = 7

print(args)

Namespace(data_path_1_SFT='./drive/MyDrive/Models/instagram_chatgpt_F.jsonl', model_name='./drive/MyDrive/Models/final_SFT_F', max_epochs=7, train_batch_size=16, output_dir='./drive/MyDrive/Models/final_SFT_F', save_steps=5000)


In [ ]:
# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{query}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "### Instruction(명령어):\n{query}\n\n### Response(응답):"
    ),
}

In [ ]:
## 모델 준비
model = AutoModelForCausalLM.from_pretrained(args.model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "skt/kogpt2-base-v2",
    padding_side="right",
    model_max_length=64,    
)
tokenizer.add_special_tokens(
    {
        "eos_token": DEFAULT_EOS_TOKEN,
        "bos_token": DEFAULT_BOS_TOKEN,
        "unk_token": DEFAULT_UNK_TOKEN,
    }
)    
tokenizer.pad_token = tokenizer.eos_token
print(tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=64, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


In [ ]:
print(f"model name : {args.model_name}")

model name : ./drive/MyDrive/Models/final_SFT_F


In [ ]:
## prepare data
from typing import Optional, Dict, Sequence
import random
import math
    
class SFT_dataset(Dataset):
    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")
        
        ## format
        pattern_instruction = 'query'  # instruction
        pattern_input = 'input'  # 내 데이터엔 input이 없다
        pattern_output = 'response'  # output

        ############################################################
        ## load dataset
        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = []
            if SFT_SAMPLE_DATA_SIZE and SFT_SAMPLE_DATA_SIZE > 0:
                random.seed(530)
                list_data_dict = random.sample(json.load(json_file), SFT_SAMPLE_DATA_SIZE)
            else:
                list_data_dict = json.load(json_file)
    
        ############################################################
        ## 데이터셋 만들기, source와 target
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]  # 템플릿 가져오기

        # 입력
        sources = []
        for example in list_data_dict:
            if example.get(pattern_input, "") != "":
                tmp = prompt_input.format_map(example)
            else:
                tmp = prompt_no_input.format_map(example)
            sources.append(tmp)

        # 출력
        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")

        if verbose:
            idx = 0
            print((sources[idx]))
            print((targets[idx]))
            print("Tokenizing inputs... This may take some time...")

        ############################################################
        # data_dict = preprocess(sources, targets, tokenizer)  # https://github.com/Beomi/KoAlpaca/blob/04704348d58b8b1c2e2638d6437a04b4e8ba1823/train.py#L124
        examples = [s + t for s, t in zip(sources, targets)]

        # source data tokenized
        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source만
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target


        ## 입력은 source, 출력은 source+target 이지만 학습은 target 부분만
        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = IGNORE_INDEX  # source 부분은 -100으로 채운다

        data_dict = dict(input_ids=input_ids, labels=labels)        
        
        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))    
        
    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        """Tokenize a list of strings."""
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]

        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )        
        
        
    def __len__(self):
        return len(self.input_ids)

    
    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)

        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

    
train_dataset = SFT_dataset(data_path_1_SFT=args.data_path_1_SFT, tokenizer=tokenizer)
eval_dataset  = None
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)


## 학습

In [ ]:
## 학습 (10min)
# training_args 수정 가능: https://github.com/Beomi/KoAlpaca/blob/main/train.sh 참고
training_args = TrainingArguments(
    output_dir=SFT_MODEL_OUTPUT_PATH, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs = args.max_epochs, # number of training epochs
    per_device_train_batch_size = args.train_batch_size, # batch size for training
    per_device_eval_batch_size = args.train_batch_size,  # batch size for evaluation
    save_steps = args.save_steps, # after # steps model is saved 
    warmup_steps = 5,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    resume_from_checkpoint=True,
    learning_rate = 5e-5,
    ignore_data_skip = True
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

trainer.train()
# trainer.train(resume_from_checkpoint=True)
trainer.save_state()
safe_save_model_for_hf_trainer(trainer=trainer, output_dir=args.output_dir)
print("SFT Model Train done!!!")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.421800
1000,2.674300
1500,2.179800
2000,1.802600
2500,1.503200
3000,1.264900
3500,1.088700


SFT Model Train done!!!


## 추론

In [ ]:
args.output_dir = './drive/MyDrive/Models/final_SFT_F'
SFT_MODEL_EVAL_DATA_PATH = './drive/MyDrive/Models/eval_response_sft_kogpt2F.txt'

with open('./drive/MyDrive/Models/eval_queries.jsonl', 'r') as r:
	eval_queries = json.load(r)

PROMPT_LIST = [query['query'] for query in eval_queries]
print(PROMPT_LIST[:10])

PROMPT_LIST = [
  '나 기분 좋아서 옷 샀어',
	'나 오늘 좋은 일 있어서 신발 샀어',
	'나 기분 안 좋아서 쇼핑다녀왔어',
	'너 기분 안 좋아보여서 아이스크림 사왔어',
	'나 돈 모아서 컴퓨터 샀어',
	'나 학교 가는 길에 교통사고 났어',
	'나 기분 안 좋아서 머리 잘랐어',
	'요즘 취업 준비하느라 힘들어',
	'오늘 늦게일어나서 대충 머리감고 나왔어',
	'걔가 너 싫어한대',
	'나 배탈 난 것 같아',
	'나 감기 걸린 것 같아',
	'코로나 걸렸어',
	'시험에 떨어졌어',
	'우울해서 아이패드 샀어',
	'나 부모님한테 혼났어',
	'이러면 아무도 너 안 좋아해',
]

['모르겠어. 약간 특수 펌 같기도 하던데', '어 업무가 너무 힘들어서 그런지 맨날 전화 와서 운다 ㅠ', '사랑해서 결혼하는데 무슨 기를 꺾어...', '건강해지는 방법이라고? 왜?', '어우 맞아요! 전 그거 보고 너무 미안해지더라고요', '아니 궁금해서 눈을 못 떼겠던데', '고등학교 때 가사 수업 하기 싫어서 선택했지', '첫째 누나 무섭다 그러고 둘째 누나는 착하다 그러더라', '요즘 엄마들은 교육열 장난 아닐 듯...', '경주에 엑스포도 있었어? 처음 들어보는데?']


In [ ]:
## 추론 테스트
generator = pipeline('text-generation', model=args.output_dir, tokenizer=tokenizer)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

list_prompt = PROMPT_LIST

list_prompt = [PROMPT_DICT['prompt_no_input'].format_map({'query' : tmp}) for tmp in list_prompt]
output_list = []
list_result = generator(list_prompt, **generation_args)
for prompt, result in zip(list_prompt, list_result):
  output = result[0]['generated_text']
  print(('#'*70))
  print(('completion: %s'%(output)))
  output_list.append(output.split("Response(응답):")[1])

# with open(SFT_MODEL_EVAL_DATA_PATH, 'w') as w:
#   w.write('\n'.join(output_list))

    

######################################################################
completion: ### Instruction(명령어):
나 기분 좋아서 옷 샀어

### Response(응답):와 정말 좋은 아이디어 같아요! 어떤 스타일의 옷을 입었는지 궁금하네요.가 입고 싶으신가요?  저 웃 사이 다 어 하 생
######################################################################
completion: ### Instruction(명령어):
나 오늘 좋은 일 있어서 신발 샀어

### Response(응답):와! 좋은 일 있었나봐요! 어떤 신발을 샀어요? 언제 사셨어요? 그리고 어떤 종류의 신발이 좋으세요? 저도 함께  어하 하 기 생
######################################################################
completion: ### Instruction(명령어):
나 기분 안 좋아서 쇼핑다녀왔어

### Response(응답):그래도 같이 쇼핑한 것 같아서 좋았겠다. 어떤 상품을 샀어? 얘기해봐. 즐거웠어! 저 어 웃  하 함 생
######################################################################
completion: ### Instruction(명령어):
너 기분 안 좋아보여서 아이스크림 사왔어

### Response(응답):아, 그러셨군요. 기분이 안 좋으신 건가요? 무슨 일 있으셨나요? 얘기해봐요. 함께 공감해드릴게요.소식 다  어 건강 
######################################################################
completion: ### Instruction(명령어):
나 돈 모아서 컴퓨터 샀어

### Response(응답):와! 정말 대단해요! 컴퓨터 값이 이렇게 비싸다니까

In [ ]:
raise Exception

# 2. RM: 좋은 글 채점기 만들기

## import (1에서 import하지 않은 상황)

In [ ]:
# ## setup(1min)
# # torch 버전 다운. torch>=2.0 에선 colosalai가 동작안함
# !pip uninstall torch -y
# !pip install torch==1.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

# import torch

# print("Torch version:{}".format(torch.__version__))
# print("cuda version: {}".format(torch.version.cuda))
# print("cudnn version:{}".format(torch.backends.cudnn.version()))

# # for ColossalAI
# !pip install colossalai==0.2.7

# # setup data
# !git clone https://github.com/airobotlab/KoChatGPT
# !mv KoChatGPT/data_kochatgpt .
# !mv KoChatGPT/img .

# %cd KoChatGPT/colossalai_ChatGPT_230319/
# !pip install .
# %cd ../../

In [ ]:
# import
import argparse

import loralib as lora
import torch
torch.cuda.empty_cache() 
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.models.bloom import BLOOMRM
from chatgpt.models.gpt import GPTRM
from chatgpt.models.opt import OPTRM
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import ColossalAIStrategy, DDPStrategy, NaiveStrategy
from datasets import load_dataset
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer

from colossalai.nn.optimizer import HybridAdam

import os
import json

# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    ),
}

## RM hyperparameter 설정

In [ ]:
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--output_dir', type=str, default=RM_MODEL_OUTPUT_PATH)
parser.add_argument('--data_path_2_RM', type=str, default=RM_DATASET_PATH, help='https://huggingface.co/datasets/fka/awesome-chatgpt-prompts/blob/main/prompts.csv')
parser.add_argument('--strategy',
                    choices=['naive', 'ddp', 'colossalai_gemini', 'colossalai_zero2'],
                    default='naive')
parser.add_argument('--model', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--pretrain', type=str, default=None)
parser.add_argument('--dataset', type=str, default='Dahoas/rm-static')
parser.add_argument('--save_path', type=str, default='rm_ckpt.pth')
parser.add_argument('--max_epochs', type=int, default=10)
parser.add_argument('--batch_size', type=int, default=4)
parser.add_argument('--lora_rank', type=int, default=0, help="low-rank adaptation matrices rank")
parser.add_argument('--max_len', type=int, default=64)  # wygo 추가

args = parser.parse_args(args=[])

# for test
args.max_epochs = 5
args.pretrain = SFT_MODEL_NAME  # pretrained 모델 가져오기
args.verbose = True

print(args)
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

In [ ]:
os.environ['RANK'] = '0'
os.environ['LOCAL_RANK'] = '0'
os.environ['WORLD_SIZE'] = '2'
os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '42043'

In [ ]:
# configure strategy
if args.strategy == 'naive':
    strategy = NaiveStrategy()
elif args.strategy == 'ddp':
    strategy = DDPStrategy()
elif args.strategy == 'colossalai_gemini':
    strategy = ColossalAIStrategy(stage=3, placement_policy='cuda')
elif args.strategy == 'colossalai_zero2':
    strategy = ColossalAIStrategy(stage=2, placement_policy='cuda')
else:
    raise ValueError(f'Unsupported strategy "{args.strategy}"')

In [ ]:
# customizing, https://github.com/hpcaitech/ColossalAI/blob/2e16f842a9e5b1fb54e7e41070e9d2bb5cd64d7c/applications/ChatGPT/chatgpt/nn/gpt_rm.py#L29
from typing import Optional

import torch.nn as nn
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

# from ..base import RewardModel
from chatgpt.models.base import RewardModel


class GPTRM_custom(RewardModel):
    """
    GPT Reward model.
    Args:
        pretrained (str): Pretrained model name or path.
        config (GPT2Config): Model config.
        checkpoint (bool): Enable gradient checkpointing.
        lora_rank (int): Rank of the low-rank approximation.
        lora_train_bias (str): LoRA bias training mode.
    """

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))  # wygo 추가!!!
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        
        # model = model.resize_token_embeddings(len(tokenizer))

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        # 추가, 230421    
        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained
        
    # 추가, 230421, config.json을 생성하기 위해 추가
    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

In [ ]:
# configure model, tokenizer
with strategy.model_init_context():
    # load pretrained gpt2    
    if args.model == 'gpt2':
#         tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        # tokenizer = AutoTokenizer.from_pretrained(args.pretrain)
        tokenizer = AutoTokenizer.from_pretrained(args.pretrain, padding_side="right", model_max_length=512)
        tokenizer.add_special_tokens(
            {
                "eos_token": DEFAULT_EOS_TOKEN,
                "bos_token": DEFAULT_BOS_TOKEN,
                "unk_token": DEFAULT_UNK_TOKEN,
            }
        )
        tokenizer.pad_token = tokenizer.eos_token
        model = GPTRM_custom(pretrained=args.pretrain, lora_rank=args.lora_rank, tokenizer=tokenizer).cuda()

    elif args.model == 'bloom':
        model = BLOOMRM(pretrained=args.pretrain, lora_rank=args.lora_rank).cuda()
        tokenizer = BloomTokenizerFast.from_pretrained(args.pretrain)
    
    elif args.model == 'opt':
        model = OPTRM(pretrained=args.pretrain, lora_rank=args.lora_rank).cuda()
        tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")      
    
    else:
        raise ValueError(f'Unsupported model "{args.model}"')
    
    
    # model.resize_token_embeddings(len(tokenizer))

In [ ]:
# make ranking data to chosen, rejetced data
with open(args.data_path_2_RM, "r", encoding='utf-8') as json_file:
    list_data_dict = json.load(json_file)
    if args.verbose:
        print('## data check ##')
        print((list_data_dict[0]))
        
total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    # data 1) 0 VS 1
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)


    # data 2) 0 VS 2
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    # data 1) 1 VS 2
    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)
    
    
    
    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[40])

In [ ]:
# prepare for data and dataset
import random
import math

random.seed(230319)
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

train_data_len = math.floor(len(total_data_ranking2chosen) * 0.8)
train_data = total_data_ranking2chosen[:train_data_len]
eval_data = total_data_ranking2chosen[train_data_len:] 


train_dataset = RewardDataset(train_data, tokenizer, args.max_len)
eval_dataset = RewardDataset(eval_data, tokenizer, args.max_len)

print(f'train set : {len(train_data)}, eval_set : {len(eval_data)}')

# check
idx = 10
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

In [ ]:
# configure optimizer
if args.strategy.startswith('colossalai'):
    optim = HybridAdam(model.parameters(), lr=5e-5)
else:
    optim = Adam(model.parameters(), lr=5e-5)

In [ ]:
# batch_size here is expected to be C(k,2), k means # response of each prompt
# be limited with the format of dataset 'Dahoas/rm-static', we'd better use batch_size as 1
trainer = RewardModelTrainer(model=model,
                             strategy=strategy,
                             optim=optim,
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=args.batch_size,
                             max_epochs=args.max_epochs)

## 학습

In [ ]:
# train!!
trainer.fit(use_lora=args.lora_rank)

## save
# save model checkpoint after fitting on only rank0
strategy.save_model(model, os.path.join(args.output_dir, 'RM.pt'), only_rank0=True)
# save optimizer checkpoint on all ranks
strategy.save_optimizer(optim,
                        os.path.join(args.output_dir, 'RM_optim_checkpoint_%d.pt' % (torch.cuda.current_device())),
                        only_rank0=False)

model.save_pretrained(args.output_dir)  # config.json 생성

## 추론

In [ ]:
# 보상모델 체크
def inference_RM(input_text='인공지능은 인공지능 입니다'):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]
    
    print('input: %s\nreward score: %.1f'%(input_text, output_reward))
    
    return output_reward


# input_text = '한국은 대한민국 입니다'
input_text = '인공지능은 인공지능 입니다'

output_reward = inference_RM(input_text=input_text)

# Test
inference_RM(input_text='어떤 신발을 샀어?')
inference_RM(input_text='샀다고 신발을?')
inference_RM(input_text='어육 제육 키키')

In [ ]:
raise Exception

#3. PPO 사람의 피드백을 반영하여 학습

    

## Import(1, 2에서 하지 않은 경우)

In [ ]:
## setup(1min)
# torch 버전 다운. torch>=2.0 에선 colosalai가 동작안함
!pip uninstall torch -y
!pip install torch==1.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
# print("cudnn version:{}".format(torch.backends.cudnn.version()))

# for ColossalAI
# !pip install colossalai==0.2.7

# setup data
!git clone https://github.com/airobotlab/KoChatGPT
!mv KoChatGPT/data_kochatgpt .
!mv KoChatGPT/img .

# for evaluation
!pip install konlpy

%cd KoChatGPT/colossalai_ChatGPT_230319/
!pip install .
%cd ../../

Found existing installation: torch 2.0.1+cu118
Uninstalling torch-2.0.1+cu118:
  Successfully uninstalled torch-2.0.1+cu118
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 902.0 kB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1+cu116 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.1+cu116 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1+cu116 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 1.13.1+cu116 which is incompatible.
Torch version:1.13.1+cu116
cuda version: 11.6
Cloning into 'KoChatGPT'...
remote: Enumera

/content


In [ ]:
# import
import argparse
from copy import deepcopy

import pandas as pd
import torch
torch.cuda.empty_cache()
from chatgpt.models.base import RewardModel
from chatgpt.models.bloom import BLOOMActor, BLOOMCritic
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.models.opt import OPTActor, OPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import ColossalAIStrategy, DDPStrategy, NaiveStrategy
from torch.optim import Adam
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer

from colossalai.nn.optimizer import HybridAdam

## wy 추가
import json
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

## clossalAI error 해결
os.environ['RANK'] = '0'
os.environ['LOCAL_RANK'] = '0'
os.environ['WORLD_SIZE'] = '2'
os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '42043'

# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    ),
}

/usr/local/lib/python3.10/dist-packages/torch/library.py:130: UserWarning: Overriding a previously registered kernel for the same operator and the same dispatch key
  operator: aten::index.Tensor(Tensor self, Tensor?[] indices) -> Tensor
    registered at aten/src/ATen/RegisterSchema.cpp:6
  dispatch key: Meta
  previous kernel: registered at ../aten/src/ATen/functorch/BatchRulesScatterOps.cpp:1053
       new kernel: registered at /dev/null:241 (Triggered internally at ../aten/src/ATen/core/dispatch/OperatorEntry.cpp:150.)
  self.m.impl(name, dispatch_key, fn)


## Hyperparameter 설정

In [ ]:
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--data_path_3_PPO', type=str, default=PPO_DATASET_PATH)
parser.add_argument('--output_dir', type=str, default=PPO_MODEL_OUTPUT_PATH)
parser.add_argument('--strategy',
                    choices=['naive', 'ddp', 'colossalai_gemini', 'colossalai_zero2'],
                    default='naive')
parser.add_argument('--model', type=str, default='gpt2', choices=['gpt2', 'bloom', 'opt'])
parser.add_argument('--pretrain', type=str, default='skt/kogpt2-base-v2')
parser.add_argument('--num_episodes', type=int, default=50)
parser.add_argument('--max_timesteps', type=int, default=200)
parser.add_argument('--steps', type=int, default=80)
parser.add_argument('--update_timesteps', type=int, default=2)
parser.add_argument('--max_epochs', type=int, default=2)
parser.add_argument('--train_batch_size', type=int, default=16)
parser.add_argument('--lora_rank', type=int, default=0, help="low-rank adaptation matrices rank")
parser.add_argument('--max_length', type=int, default=64)
args = parser.parse_args(args=[])

## 이곳 수정!!
args.pretrain_actor = SFT_MODEL_PATH  # SFT 모델 가져오기
args.pretrain_critic = RM_MODEL_PATH  # RM 모델 가져오기
# args.pretrain_actor = args.pretrain
# args.pretrain_critic = args.pretrain

args.num_episodes = 15
args.max_timesteps = 80
args.max_epochs   = 1

print(args)
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

Namespace(data_path_3_PPO='./drive/MyDrive/Models/facebook_ppo.jsonl', output_dir='./drive/MyDrive/Models/final_PPO_light_F', strategy='naive', model='gpt2', pretrain='skt/kogpt2-base-v2', num_episodes=15, max_timesteps=80, steps=80, update_timesteps=2, max_epochs=1, train_batch_size=16, lora_rank=0, max_length=64, pretrain_actor='./drive/MyDrive/Models/final_SFT_F', pretrain_critic='./drive/MyDrive/Models/final_RM')


In [ ]:
# configure strategy
if args.strategy == 'naive':
    strategy = NaiveStrategy()
elif args.strategy == 'ddp':
    strategy = DDPStrategy()
elif args.strategy == 'colossalai_gemini':
    strategy = ColossalAIStrategy(stage=3, placement_policy='cuda')
elif args.strategy == 'colossalai_zero2':
    strategy = ColossalAIStrategy(stage=2, placement_policy='cuda')
else:
    raise ValueError(f'Unsupported strategy "{args.strategy}"')

In [ ]:
# configure model, tokenizer
with strategy.model_init_context():
    actor = GPTActor(pretrained=args.pretrain_actor, lora_rank=args.lora_rank).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained=args.pretrain_critic, lora_rank=args.lora_rank).to(torch.cuda.current_device())
    # tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    # tokenizer.pad_token = tokenizer.eos_token
    tokenizer = AutoTokenizer.from_pretrained(args.pretrain, padding_side="right", model_max_length=512)
    tokenizer.add_special_tokens(
        {
            "eos_token": DEFAULT_EOS_TOKEN,
            "bos_token": DEFAULT_BOS_TOKEN,
            "unk_token": DEFAULT_UNK_TOKEN,
        }
    )    
    tokenizer.pad_token = tokenizer.eos_token


    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

In [ ]:
# setting the models
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = strategy.prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [ ]:
# prepare data
with open(args.data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

print(len(list_prompt))
print('\n\n\n')
print(tokenize_fn('I want you to act as a linux terminal.'))

119163




{'input_ids': tensor([[10054, 18896, 16740, 19490, 15949, 20938, 44696,   458, 46654, 13612,
         13008,  9610, 30259, 10448, 36305,  9610,  9776,   389]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


## 학습

In [ ]:
# configure trainer
trainer = PPOTrainer(strategy,
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=args.max_epochs,
                     train_batch_size=args.train_batch_size,
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

## train!
trainer.fit(list_prompt,  # 입력 prompt
            num_episodes=args.num_episodes,
            max_timesteps=args.max_timesteps,
            update_timesteps=args.update_timesteps)

## save
# save model checkpoint after fitting on only rank0
strategy.save_model(actor, os.path.join(args.output_dir, 'actor.pt'), only_rank0=True)
# save optimizer checkpoint on all ranks
strategy.save_optimizer(actor_optim,
                        os.path.join(args.output_dir, 'actor_optim_checkpoint_%d.pt' % (torch.cuda.current_device())),
                        only_rank0=False)

Episode [15/15]: 100%|██████████| 80/80 [03:19<00:00,  2.49s/it]


## 추론

In [ ]:
## inference
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=args.max_length,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print('#' * 70)
    print(output)
    return output


list_prompt = PROMPT_LIST

list_prompt = [PROMPT_DICT['prompt_no_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text)

NameError: ignored

In [ ]:
raise Exception

# PPO까지 모두 완료된 모델로 추론하기

맨 위의 상수 설정 필수

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import json

SFT_MODEL_NAME = './drive/MyDrive/Models/final_SFT_F'
PPO_MODEL_PATH = './drive/MyDrive/Models/final_PPO_light_F'
PPO_MODEL_EVAL_DATA_PATH = './drive/MyDrive/Models/eval_response_kogpt2F.txt'

with open('./drive/MyDrive/Models/eval_queries.jsonl', 'r') as r:
	eval_queries = json.load(r)

PROMPT_LIST = [query['query'] for query in eval_queries]
print(PROMPT_LIST[:10])

# PROMPT_LIST = [
#   '나 기분 좋아서 옷 샀어',
# 	'나 오늘 좋은 일 있어서 신발 샀어',
# 	'나 기분 안 좋아서 쇼핑다녀왔어',
# 	'너 기분 안 좋아보여서 아이스크림 사왔어',
# 	'나 돈 모아서 컴퓨터 샀어',
# 	'나 학교 가는 길에 교통사고 났어',
# 	'나 기분 안 좋아서 머리 잘랐어',
# 	'요즘 취업 준비하느라 힘들어',
# 	'오늘 늦게일어나서 대충 머리감고 나왔어',
# 	'걔가 너 싫어한대',
# 	'나 배탈 난 것 같아',
# 	'나 감기 걸린 것 같아',
# 	'코로나 걸렸어',
# 	'시험에 떨어졌어',
# 	'우울해서 아이패드 샀어',
# 	'나 부모님한테 혼났어',
# 	'이러면 아무도 너 안 좋아해',
# ]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['모르겠어. 약간 특수 펌 같기도 하던데', '어 업무가 너무 힘들어서 그런지 맨날 전화 와서 운다 ㅠ', '사랑해서 결혼하는데 무슨 기를 꺾어...', '건강해지는 방법이라고? 왜?', '어우 맞아요! 전 그거 보고 너무 미안해지더라고요', '아니 궁금해서 눈을 못 떼겠던데', '고등학교 때 가사 수업 하기 싫어서 선택했지', '첫째 누나 무섭다 그러고 둘째 누나는 착하다 그러더라', '요즘 엄마들은 교육열 장난 아닐 듯...', '경주에 엑스포도 있었어? 처음 들어보는데?']


In [ ]:
# import
import argparse

import torch
torch.cuda.empty_cache()
from chatgpt.models.gpt import GPTActor
from transformers import AutoTokenizer
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer
import os

# data config
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input":
    (
     "### Instruction(명령어):\n{prompt}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
     ),
    "prompt_no_input":
    (
     "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"),
}

In [ ]:
# define argment
parser = argparse.ArgumentParser()
parser.add_argument('--model',
                    default='gpt2')
# We suggest to use the pretrained model from HuggingFace, use pretrain to configure model
parser.add_argument('--pretrain', type=str, default=None)
parser.add_argument('--model_path', type=str, default=None)
parser.add_argument('--input',
                    type=str,
                    default='Question: How are you ? Answer:')
parser.add_argument('--max_length', type=int, default=250)
args_inference = parser.parse_args([])

args_inference.model = 'gpt2'
args_inference.pretrain = SFT_MODEL_NAME
args_inference.model_directory = PPO_MODEL_PATH
args_inference.model_path = os.path.join(args_inference.model_directory, 'actor.pt')

# configure model, tokenizer
actor = GPTActor(pretrained=args_inference.pretrain).to(torch.cuda.current_device())
# tokenizer = GPT2Tokenizer.from_pretrained(args_inference.pretrain)
# tokenizer.pad_token = tokenizer.eos_token
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2",
                                          padding_side="right",
                                          model_max_length=512)
tokenizer.add_special_tokens({
    "eos_token": DEFAULT_EOS_TOKEN,
    "bos_token": DEFAULT_BOS_TOKEN,
    "unk_token": DEFAULT_UNK_TOKEN,
})
tokenizer.pad_token = tokenizer.eos_token

state_dict = torch.load(args_inference.model_path, map_location='cpu');
actor.model.load_state_dict(state_dict);

actor.eval();

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
## inference
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=args_inference.max_length,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print('#' * 70)
    print(output)
    return output


list_prompt = PROMPT_LIST
list_prompt = [PROMPT_DICT['prompt_no_input'].format_map({'prompt': tmp}) for tmp in list_prompt]
output_list = []

for input_text in list_prompt:
    output = generation(input_text)
    output_list.append(output.split("Response(응답):")[1])

with open(PPO_MODEL_EVAL_DATA_PATH, 'w') as w:
  w.write('\n'.join(output_list))

######################################################################
### Instruction(명령어):
모르겠어. 약간 특수 펌 같기도 하던데

### Response(응답):응, 맞아. 그런 느낌이야. 너도 그런 특수 시술을 받고 싶어해?
######################################################################
### Instruction(명령어):
어 업무가 너무 힘들어서 그런지 맨날 전화 와서 운다 ᅲ

### Response(응답):그래서 조금은 지칠 때가 있잖아. 어떻게 하면 좀 쉬어질까?
######################################################################
### Instruction(명령어):
사랑해서 결혼하는데 무슨 기를 꺾어...

### Response(응답):사랑한다는 의지와 의지가 있으시군요! 어떤 상황에서 그런 인연을
######################################################################
### Instruction(명령어):
건강해지는 방법이라고? 왜?

### Response(응답):아마 도움이 필요하면 같이 할 것 같아.
######################################################################
### Instruction(명령어):
어우 맞아요! 전 그거 보고 너무 미안해지더라고요

### Response(응답):그랬군요. 그러면 나중에 다른 영화 추천해주세요!
######################################################################
### Instruction(명령어):
아니 궁금해서 눈을 못 떼겠던데

### Response(응답):아, 그렇군요. 궁금한 것이 있으면 언제든지 말해봐요. 함께 고민해

## BLEU 스코어 측정

code로 작성하는 것보단, 추론 결과를 전달해줘서 한 코드에서 돌리는 것이 좋겠다!
bleu_score vs sententce_blue?